In [1]:
import pandas as pd
import ast
import itertools


f = pd.read_csv("ticdata2000.txt",delimiter="\t",header=None)
f.loc[:,30:41].to_csv('myData.csv',header=False, index=False)

colnames = []
for i in range (31,43):
    colnames.append('a'+str(i))
data = pd.read_csv('myData.csv',names=colnames, header=None)
 
rows = data.shape[0]
datacopy1 = data.copy()

In [2]:
support=input("minimum support: ")
support=float(support)
minConfidence=input("minimum confidence: ")
minConfidence=float(minConfidence)

minimum support: 0.2
minimum confidence: 0.5


In [3]:
uniqueValues = []
i = 0 
for attr in colnames:
    column_values = data[attr].values.ravel()
    uniqueValues.append(pd.unique(column_values))
    i = i+1
#print(uniqueValues)


allValues = []
i = 0 
for attr in colnames:
    valueCollector = []
    for unique in uniqueValues[i]:
        valueCollector.append(attr+'_'+str(unique))
    allValues.append(valueCollector)
    i = i+1
#print(allValues)


totalNumAttr = 0
for i in range(len(allValues)):
    totalNumAttr += len(allValues[i])
    
    
for attr in colnames:
    i = colnames.index(attr)
    lengthAttrValues = len(allValues[i])
    for l in range(lengthAttrValues):
        index = data[:][attr] == int(allValues[i][l][-1])
        changed = data[index]
        datacopy1.loc[changed.index,attr] = allValues[i][l]

In [4]:
datacopy1

,a31,a32,a33,a34,a35,a36,a37,a38,a39,a40,a41,a42
0,a31_8,a32_8,a33_0,a34_1,a35_8,a36_1,a37_0,a38_4,a39_5,a40_0,a41_0,a42_4
1,a31_7,a32_7,a33_1,a34_2,a35_6,a36_3,a37_2,a38_0,a39_5,a40_2,a41_0,a42_5
2,a31_2,a32_7,a33_0,a34_2,a35_9,a36_0,a37_4,a38_5,a39_0,a40_0,a41_0,a42_3
3,a31_4,a32_9,a33_0,a34_0,a35_7,a36_2,a37_1,a38_5,a39_3,a40_0,a41_0,a42_4
4,a31_5,a32_6,a33_2,a34_1,a35_5,a36_4,a37_0,a38_0,a39_9,a40_0,a41_0,a42_6
...,...,...,...,...,...,...,...,...,...,...,...,...
5817,a31_0,a32_5,a33_1,a34_3,a35_5,a36_4,a37_4,a38_3,a39_3,a40_0,a41_0,a42_3
5818,a31_6,a32_6,a33_1,a34_2,a35_6,a36_3,a37_0,a38_9,a39_0,a40_0,a41_0,a42_4
5819,a31_2,a32_4,a33_0,a34_5,a35_8,a36_1,a37_5,a38_3,a39_1,a40_1,a41_0,a42_3
5820,a31_7,a32_5,a33_4,a34_0,a35_9,a36_0,a37_0,a38_5,a39_4,a40_0,a41_0,a42_4


In [5]:
#input a list of values AND number of combinations you want 
# The output will be all the possible combinatinos
#Example :   INPUT ['a','b','c','d']   ... OUTPUT ('a','b'),('a','c'),('a','d'),('b','c'),('b','d'),('c','d')
def TupleGenerator(List,Tuple):
    output = []
    T = Tuple
    for l in itertools.combinations(List, T):
            output.append(l)
    return output






#input list of strings of lists ... example ["['a41_0', 'a33_0', 'a40_0']", "['a41_0', 'a40_0', 'a42_3']"]
#output lists of lists  ... example [['a41_0', 'a33_0', 'a40_0'], ['a41_0', 'a40_0', 'a42_3']]

def listOfStrings_to_listOfLists (listOfStrings,length):
    listOfLists = []
    for i in range(length):
        listOfLists.append(ast.literal_eval(listOfStrings[i]))  
    return (listOfLists)


#freq=["['a37_0', 'a40_0']", "['a35_9', 'a36_0']", "['a35_7', 'a36_2']", "['a35_7', 'a36_2']"]

#input any list of lists  AND Outputs a List of all unique values that was in those lists 
def listCreator(stringOfLists,length):
    ft = []
    for i in range(length):
        ft.append(ast.literal_eval(stringOfLists[i]))

    bucket = []
    for f in ft:
        for i in range(len(f)):
            if f[i] not in bucket:
                bucket.append(f[i])
    return(bucket)
#print(listCreator(freq,len(freq)))





def getSupport (inputDicItems):
    ListOfItemsWithMinSupport = []
    dicOfSupports= {}
    for key in inputDicItems:
        if (inputDicItems[key]/rows) > support:
            ListOfItemsWithMinSupport.append(key)
            dicOfSupports[key]=inputDicItems[key]
    return ListOfItemsWithMinSupport,dicOfSupports





def Confidence (intersection,left,right):
    confidence = intersection/left
    return confidence
                    
    
    
    
def Lift (intersection,left,right):
    return( (intersection/(left*right)) * rows   )

def Leverage (intersection,left,right):
    return(  ((intersection)-(left*right/rows))/rows  )


def printStuff (confidence,left,right,first,second):
    print(first, "  -->  ",second)
    print("Confidence = ",confidence)
    print("Lift = ",Lift (intersection,left,right))
    print("Leverage = ",Leverage (intersection,left,right))

In [6]:
itemCount =  {}
for i in range(len(colnames)):
    attr = colnames[i]
    for a in allValues[i]:
        occurence = datacopy1.loc[ (datacopy1[attr]==a)].shape[0]
        if occurence != 0:
            itemCount[a] = occurence
            
            
l,d = getSupport (itemCount)

In [7]:
listOfSupports = []

listOfSupports.append(d)

In [8]:
if len(l) > 0:
    counter = 2 # start with pairs 
    TupleValues = l 
    FINAL = []
    while(True):
        if len(l) == 0:
            counter -= 1
            break
        if counter > 2:
            TupleValues = listCreator(l,len(l))
        TupleCount =  {} #will store the temporary tuples "pairs,Triples,etc."
        Tuples =TupleGenerator(TupleValues,counter)
        print(counter)
        for Tuple in Tuples:
            length = len(Tuple)
            TupleIndexBucket = []
            TupleIndexBucket.append(Tuple[0])
            TupleIndexBucket.append(Tuple[1])

            index = (datacopy1[Tuple[0][0:3]]==Tuple[0]) & (datacopy1[Tuple[1][0:3]]==Tuple[1])
            if length > 2:
                i = 2
                while(i<length):
                    TupleIndexBucket.append(Tuple[i])
                    index &= (datacopy1[Tuple[i][0:3]]==Tuple[i])
                    i += 1

            occurence = datacopy1[index].shape[0]
            TupleIndexBucket.sort()
            if occurence != 0:
                TupleCount[str(TupleIndexBucket)] = occurence
        l,d =getSupport (TupleCount)
        listOfSupports.append(d)
        print("DONE")

        counter += 1

2
DONE
3
DONE
4
DONE


In [9]:
lastNumberofSupportedItems = counter - 1

In [10]:
listOfSupports[lastNumberofSupportedItems-1]

{"['a33_0', 'a40_0', 'a41_0']": 1201,
 "['a40_0', 'a41_0', 'a42_3']": 1270,
 "['a35_7', 'a36_2', 'a41_0']": 1338}

In [11]:
frequentItems =[]
for key in listOfSupports[lastNumberofSupportedItems-1]:
    frequentItems.append(key)
frequentItems = listOfStrings_to_listOfLists (frequentItems,len(frequentItems))

In [12]:
frequentItems

[['a33_0', 'a40_0', 'a41_0'],
 ['a40_0', 'a41_0', 'a42_3'],
 ['a35_7', 'a36_2', 'a41_0']]

In [14]:
limit = lastNumberofSupportedItems-1
for f in frequentItems:
    search = str(list(f))
    intersection = listOfSupports[lastNumberofSupportedItems-1].get(search)
    inferenceSupport = {}
    values = []
    temp = []
    for search in f:
        support = listOfSupports[0].get(search)
        inferenceSupport[search] = support
        temp.append(search)
    values.append(temp)
    for i in range(2,lastNumberofSupportedItems):
        comb = TupleGenerator(f,i)
        temp = []
        for c in comb:
            search = str(list(c))
            support = listOfSupports[i-1].get(search)
            temp.append(search)
            inferenceSupport[search] = support
        values.append(temp)
    for v in range(1,len(values)):
        values[v] = listOfStrings_to_listOfLists(values[v],len(values[v]))
    
    if limit % 2 == 0:
        Range = limit//2
    else:
        Range = (limit//2)+1
    for i in range(Range):
        for first in values[i]:
            for second in values[limit-1-i]:
                leftindex = str(first)
                rightindex = str(second)
                left = inferenceSupport[leftindex]
                right = inferenceSupport[rightindex]
                confidence = Confidence (intersection,inferenceSupport[leftindex],inferenceSupport[rightindex])
            
                left1index = rightindex
                right1index = leftindex
                left1 = inferenceSupport[left1index]
                right1 = inferenceSupport[right1index]
            
                output = 1
                if i != 0:
                    for it in first:
                        if it in second:
                            output = 0
                            break
                else:
                    if first in second:
                        output = 0
                if output == 1:
                    if confidence > minConfidence:
                        printStuff (confidence,left,right,first,second)
                    if i != (limit//2):
                        confidence = Confidence (intersection,inferenceSupport[left1index],inferenceSupport[right1index])
                        if confidence > minConfidence:
                            printStuff (confidence,left1,right1,second,first)


a33_0   -->   ['a40_0', 'a41_0']
Confidence =  0.6477885652642934
Lift =  1.2308828416999726
Leverage =  0.03869418891207505
['a33_0', 'a41_0']   -->   a40_0
Confidence =  0.719592570401438
Lift =  1.290655559111883
Leverage =  0.04645570804825771
['a33_0', 'a40_0']   -->   a41_0
Confidence =  0.9584996009577015
Lift =  1.138854015668518
Leverage =  0.025151343752201596
['a41_0', 'a42_3']   -->   a40_0
Confidence =  0.7142857142857143
Lift =  1.28113722383593
Leverage =  0.04786898532568335
['a40_0', 'a42_3']   -->   a41_0
Confidence =  0.9650455927051672
Lift =  1.1466317225978537
Leverage =  0.027895586942573578
a42_3   -->   ['a40_0', 'a41_0']
Confidence =  0.6573498964803313
Lift =  1.249050619226008
Leverage =  0.043494977118620756
a35_7   -->   ['a36_2', 'a41_0']
Confidence =  0.885506287227002
Lift =  3.8530774321641297
Leverage =  0.17017263908879962
['a36_2', 'a41_0']   -->   a35_7
Confidence =  1.0
Lift =  3.8530774321641297
Leverage =  0.17017263908879962
a36_2   -->   ['a35